# Implementing (deterministic) finite-state automata

While n-gram models are very useful in language technology, they are also very limited in their capabilities.
If something involves more than *n* consecutive symbols, an n-gram model has a hard time handling it.
**Finite-state automata** (FSAs) are a valuable upgrade.
They can be used for part-of-speech tagging, stemming and lemmatization, word prediction, and so on.
FSAs can do everything an n-gram model can do, and more.

So why use n-gram models if FSAs are so much better?
Well, FSAs are more powerful, but they are also harder to work with.
Their implementation is a bit trickier, and they are much harder to learn automatically from corpora.
The last part is often the most important for language technology, so n-gram models are preferred because learning is so straightforward with them (just compute the n-gram counts).

Be that as it may, FSAs are an essential part of computational linguistics and you need to have a good understanding of how they work.
There's a lot of deep theory and math here, but that is best left for a separate course.
In this unit, we only care about the implementation of **deterministic FSAs**.

## Lecture notes reminder

I assume that you've already studied the lecture notes on FSAs so that you have a rough idea of how they work.
But just so that we're all on the same page, here's a few terminological reminders.

An FSA is **deterministic** iff:

1. it has exactly one start state (no more, no less), and
1. no state may have two outgoing arcs with the same label.

Keep in mind, though, that a deterministic automaton may have multiple final states.
An FSA **recognizes** or **accepts** a string iff the string describes a path from an initial state to a final state.

## A simple implementation

Implementing a finite-state automaton is actually very simple, but it requires us to look beyond the pretty pictures.
While it is very intuitive to depict an FSA as a graph of nodes that are connected by labeled arcs, this format isn't very helpful for a Python implementation.
What's a node, and what is an arc?
Just like with prefix trees, our actual data structure will differ quite a bit from the intuitive depicition.

If you ask a mathematician or computer scientist what an FSA actually is, they will tell you the following:

> An FSA consists of three components.
> 1. a unique initial state `I`,
> 1. a set `F` of final states,
> 1. a set `T` of transitions, each one of which is of the form `(current_state, label, new_state)`.

For instance, our toy automaton above is mathematically described as follows:

```python
I = 1
F = {4, 5}
T = {(1, "John", 2),
     (1, "Sue", 2),
     (2, "doubt", 3),
     (2, "left", 4),
     (2, "really", 2),
     (2, "thinks", 3),
     (3, "that", 1),
     (4, "Bill", 5)}
```

It would be very easy to use exactly the same format in Python.
Then `I`, `F`, and `T` are just variables, with `I` pointing to an integer that is the name of some state, `F` being a set of integers (i.e. states), and `T` a set of tuples (remember that tuples are immutable and thus hashable, so they can be collected in a Python set).

But this isn't a very useful format.
To see how, suppose that we want to write a function `accepts` that returns `True` if the FSA recognizes the sentence, and `False` otherwise.
How would we do this?
We could try the option below.

In [ ]:
def accepts(sentence, I, F, T):
    """Test if FSA accepts sentence.
    
    Arguments
    ---------
    sentence: list of strings
        tokenized sentence
    I: integer
        initial state of FSA
    F: set of integers
        set of FSA's final states
    T: set of triples
        FSA transitions, in form (current_state, label, new_state)
        
    Returns
    -------
    bool
    """
    # set current state
    cs = I
    # iterate over sentence and follow along in automaton
    for word in sentence:
        cs = next_state(cs, word, T)
    # did we make it to a final state?
    return True if cs in F else False


def next_state(cs, word, T):
    """Return state reached via word from current state"""
    # list of arcs leaving from cs
    transitions = [(label, next_state)
                   for (x, label, next_state) in T if x == cs]
    # return state reached via word, if it exists
    for label, next_state in transitions:
        if label == word:
            return next_state
    # didn't find any arc with word as label
    return False

Let's see if this code works with a very simple automaton that only recognizes *John likes Bill*.

In [ ]:
I = 1
F = {4}
T = {(1, "John", 2),
     (2, "likes", 3),
     (3, "Bill", 4)}

In [ ]:
sentence = ["John", "likes", "Bill"]
accepts(sentence, I, F, T)

In [ ]:
sentence = ["John", "likes", "Sue"]
accepts(sentence, I, F, T)

Alright, the code words as desired, but it is clunky.
Why do we have to pass in `I`, `F`, and `T` separately?
This obfuscates that the function operates with two basic objects, a list of strings and an FSA.
But this only works if we have a single object for FSAs, instead of three.
Fortunately, this is easily fixed with a dictionary.

In [ ]:
# defining the example FSA as a dictionary
{"I": 1,
 "F": {4},
 "T": {(1, "John", 2),
       (2, "likes", 3),
       (3, "Bill", 4)}}

In [ ]:
# defining accepts with only two arguments
def accepts(sentence, fsa):
    """Test if FSA accepts sentence.
    
    The FSA must be a dictionary of the form
    
    {'I': initial_state,
     'F': {final_state1, final_state2, ...},
     'T': set of (current_state, label, next_state) tuples}
    
    Arguments
    ---------
    sentence: list of strings
        tokenized sentence
    fsa: dict
        finite-state automaton
        
    Returns
    -------
    bool
    """
    # set current state
    cs = fsa["I"]
    # iterate over sentence and follow along in automaton
    for word in sentence:
        cs = next_state(cs, word, fsa["T"])
    # did we make it to a final state?
    return True if cs in fsa["F"] else False


def next_state(cs, word, T):
    """Return state reached via word from current state"""
    # list of arcs leaving from cs
    transitions = [(label, next_state)
                   for (x, label, next_state) in T if x == cs]
    # return state reached via word, if it exists
    for label, next_state in transitions:
        if label == word:
            return next_state
    # didn't find any arc with word as label
    return False

As you can see, the code still produces the same results.

In [ ]:
new_fsa = {"I": 1, "F": {4}, "T": {(1, "John", 2),
                                   (2, "likes", 3),
                                   (3, "Bill", 4)}}

In [ ]:
sentence = ["John", "likes", "Bill"]
accepts(sentence, new_fsa)

In [ ]:
sentence = ["John", "likes", "Sue"]
accepts(sentence, new_fsa)

So this is a slight improvement in that it is now less of a hassle to use the function if multiple automata are defined.

In [ ]:
import re

fsa1 = {"I": 1, "F": {4}, "T": {(1, "John", 2),
                                (2, "likes", 3),
                                (3, "Bill", 4)}}
fsa2 = {"I": 1, "F": {3}, "T": {(1, "Sue", 2),
                                (2, "slept", 3),
                                (2, "snored", 3),
                                (2, "knows", 1)}}


sentences = ["John likes Bill", "Sue knows Sue snored", "John likes Sue"]
for sentence in sentences:
    tokenized = re.findall(r"\w+", sentence)
    for fsa in [fsa1, fsa2]:
        status = "well-formed" if accepts(tokenized, fsa) else "ill-formed"
        print(f"\"{sentence}\" is {status}")
    print("="*20)

Doing the same with `I`, `F`, and `T` as parameters would have been more tedious.
If you want to see how exactly one could make it work, see the expansion unit.

So implementing an FSA as a dictionary is an improvement over the definition with three separate variables.
But the current data structure is still not optimal as it makes our code slower than it needs to be.
Let's look at that in detail.

## Performance analysis

Here's the current code for our `accepts` function, including its helper function `next_state`.

In [ ]:
# defining accepts with only two arguments
def accepts(sentence, fsa):
    """Test if FSA accepts sentence.
    
    The FSA must be a dictionary of the form
    
    {'I': initial_state,
     'F': {final_state1, final_state2, ...},
     'T': set of (current_state, label, next_state) tuples}
    
    Arguments
    ---------
    sentence: list of strings
        tokenized sentence
    fsa: dict
        finite-state automaton
        
    Returns
    -------
    bool
    """
    # set current state
    cs = fsa["I"]
    # iterate over sentence and follow along in automaton
    for word in sentence:
        cs = next_state(cs, word, fsa["T"])
    # did we make it to a final state?
    return True if cs in fsa["F"] else False


def next_state(cs, word, T):
    """Return state reached via word from current state"""
    # list of arcs leaving from cs
    transitions = [(label, next_state)
                   for (x, label, next_state) in T if x == cs]
    # return state reached via word, if it exists
    for label, next_state in transitions:
        if label == word:
            return next_state
    # didn't find any arc with word as label
    return False

The `accepts` function is pretty simple.
Setting `cs` to `fsa["I"]` costs virtually nothing.
The function then iterates over the sentence, look at each word once.
This, too, is optimal --- how else could you possibly do it?
There is no way to tell if a sentence is recognized by an automaton without looking at the words in the sentence, one after the other.
If `accepts` looped over the sentence multiple times, there would be room for optimization, but one loop is the best we can do.
The `return` line is also very fast.
So in sum, `accepts` looks like a reasonable piece of code.

But keep in mind that `accepts` calls the `next_state` function once for each word in the sentence.
If `next_state` is inefficient, so is `accepts`.
And unfortunately, `next_state` is very ineffcient.
The first step is to calculate the possible transitions from the current state via a list expression.
If the automaton has many transitions, this is a costly step:

1. Python has to look at each triple in the transitions set and check if the first component matches the current state.
   If an automaton has 10,000 transitions (which isn't unusual in computational linguistics), this means 10,000 checks already at the beginning of the word.
1. Python then builds a new list that only contains the relevant transitions.
   Again, this list might be large, and constructing large lists is costly.
   It costs both time and memory.
1. We then iterate over this new list of pairs and check if the first component of each pair matches the current word.
   Another iteration over a potentially long list means another loss of time.
   
As you can see, even one call of `next_state` can already be very costly.
If the automaton has 10,000 transitions and 1,000 of those start with current state, `next_state` will look at 11,000 items (10,000 in the first loop, then 1,000 in the second).
But `accepts` calls `next_state` for each word in `sentence`!
This is slow, slow, slow.
In a sentence with 10 words, Python might perform over 100,000 look-ups.

Let's see how we could fix this.
There's multiple options of increasing efficiency.

### Option 1: Eliminate the second `for` loop

One solution is to use only one `for`-loop instead of two.

In [ ]:
def next_state_single_loop(cs, word, T):
    """Return state reached via word from current state"""
    # list of states reachable from cs via word
    reachable = [next_state
                 for (x, y, next_state) in T
                 if x == cs and y == word]
    # return state reached via word, if it exists, else False
    return reachable[0] if reachable else False

Here we immediately construct a list of reachable states while iterating over the list of all transitions.
If the list is empty, we return `False`, and otherwise its first element.

In the best case, this cuts time in half: if there are 10,000 transitions and all 10,000 starts in the current state, then `next_state_single_loop` will only look at 10,000 items instead of 20,000.
But in practice, a single state doesn't have all that many outgoing arcs, so cutting out the second loop is not as useful as one might think.

### Option 2: Don't construct a list

If you think about it, what's even the point of having a list in `next_state_single_loop`?
Recall that we assume that the FSA is deterministic, so at most one state can be reached from the current state via `word`.
In that case, we don't need a list.
A simple `for`-loop will get the job done in a fashion that's both simpler and faster.

In [ ]:
def next_state_nolist(cs, word, T):
    """Return state reached via word from current state"""
    for x, y, next_state in T:
        if x == cs and y == word:
            return next_state
    return False

This solution has two advantages.
First, we no longer need to construct a list.
Second, we return as soon as the first (and only) match is found.
So if, say, we find the next state during iteration 3 and T contains 10,000 transitions, we don't need to look at the remaining 9,997 transitions.
Now of course sometimes we might not find the next state until the very last transition, in which case we don't save any time compared to looking at all transitions.
But this will happen about as often as the very first transition will be the right one.
On average, then, we need to look at roughly 5000 transitions to find the next state.
This is yet another reduction by half.
We went from 20,000 to 10,000, and now 5,000.
But we're still far from optimal.

### Option 3: Memoizing state transitions

Even with `next_state_nolist`, we still compute the next state every time the function is called.
We might easily end up recomputing previous results, e.g. with the automaton below.

In [ ]:
fsa = {"I": 1, "F": {2}, "T": (1, "b", 2),
                              (2, "a", 3),
                              (3, "n", 2)}

This automaton recognizes truncated and expanded versions of *banana*, including *ba*, *bana*, *bananananana*, and so on.
Now suppose that we want to know if the automaton accepts *bananan*.
Here's what will be computed step by step:

1. What's the next state for *b* from 1?
1. What's the next state for *a* from 2?
1. What's the next state for *n* from 3?
1. What's the next state for *a* from 2?
1. What's the next state for *n* from 3?
1. What's the next state for *a* from 2?
1. What's the next state for *n* from 3?

Notice how we keep recomputing the same things!
Steps 2, 4, 6 always yield the same result, and so do steps 3, 5, and 7.
But instead of storing the results, we keep recomputing them.
As you know by now, this is very inefficient.
So let's add some dynamic programming via memoization.

In [ ]:
def accepts(sentence, fsa):
    """Test if FSA accepts sentence.
    
    Now with dynamic programming!
    """
    # set current state
    cs = fsa["I"]
    # initialize transition memo
    memo = {}
    # iterate over sentence and follow along in automaton
    for word in sentence:
        # check memo for known transitions
        memo[cs] = memo.get(cs, {})
        reached = memo[cs].get(word)
        # get cs from memo, if possible, otherwise compute from scratch
        if reached:
            cs = reached
        else:
            cs = next_state_nolist(cs, word, fsa["T"])
            # memoize the value for future retrieval
            memo[cs][word] = cs
    # did we make it to a final state?
    return True if cs in fsa["F"] else False


def next_state_nolist(cs, word, T):
    """Return state reached via word from current state"""
    for x, y, next_state in T:
        if x == cs and y == word:
            return next_state
    return False

This code can be a lot faster since we never have to recompute any steps.
As `accepts` works its way through the string, it builds up a dictionary of the FSA's transitions.
Here's what this looks like for the *banana* FSA above, starting with `memo = {}`:

| **Query** | **In memo?** | **Value** | **Updated memo** |
| :--       | :--          | :--       | :--              |
| What's the next state for *b* from 1? | N | 2 | `{1: {"b": 2}}` |
| What's the next state for *a* from 2? | N | 3 | `{1: {"b": 2}, 2: {"a": 3}}` |
| What's the next state for *n* from 3? | N | 2 | `{1: {"b": 2}, 2: {"a": 3}, 3: {"n": 2}}` |
| What's the next state for *a* from 2? | Y | 3 | `{1: {"b": 2}, 2: {"a": 3}, 3: {"n": 2}}` |
| What's the next state for *n* from 3? | Y | 2 | `{1: {"b": 2}, 2: {"a": 3}, 3: {"n": 2}}` |
| What's the next state for *a* from 2? | Y | 3 | `{1: {"b": 2}, 2: {"a": 3}, 3: {"n": 2}}` |
| What's the next state for *n* from 3? | Y | 2 | `{1: {"b": 2}, 2: {"a": 3}, 3: {"n": 2}}` |

But all the memoization code also makes `accepts` a lot harder to read.
And look at the table above.
All the transitions of the FSA are also encoded in `memo` as a nested dictionary.
This just raises the question, why do we specify the transitions as a set of triples?
Let's just use nested dictionaries right away, saving us all the memoization effort!

## FSAs with transition dictionaries

At the very beginning of this section, we implemented an FSA through three variables:

```python
I = 1
F = {4, 5}
T = {(1, "John", 2),
     (1, "Sue", 2),
     (2, "doubt", 3),
     (2, "left", 4),
     (2, "really", 2),
     (2, "thinks", 3),
     (3, "that", 1),
     (4, "Bill", 5)}
```

We quickly realized that it makes more sense to collect them together in a single dictionary.

```python
{"I": 1,
 "F": {4},
 "T": {(1, "John", 2),
       (2, "likes", 3),
       (3, "Bill", 4)}}
```

And now we saw that encoding transitions as triples makes for an inefficient data structure that we need to work around with memoization.
Nested dictionaries are a much better choice.

```python
{"I": 1,
 "F": {4},
 "T": {1: {"John": 2},
       2: {"likes": 3},
       3: {"Bill": 4},
      },
}
```

Just look at how simpler `accept` is with this data structure.

In [ ]:
def accepts(sentence, fsa):
    """Test if FSA accepts sentence.
    
    The FSA must be a dictionary of the form
    
    {'I': initial_state,
     'F': {final_state1, final_state2, ...},
     'T': {current_state: {word: next_state}}
    
    Arguments
    ---------
    sentence: list of strings
        tokenized sentence
    fsa: dict
        finite-state automaton
        
    Returns
    -------
    bool
    """
    # set current state
    cs = fsa["I"]
    # iterate over sentence and follow along in automaton
    for word in sentence:
        cs = fsa["T"].get(cs, {}).get(word)
        if cs is None:
            return False
    # did we make it to a final state?
    return True if cs in fsa["F"] else False

And of course the function still makes the same judgments as the less efficient versions.

In [ ]:
fsa = {"I": 1,
       "F": {4},
       "T": {1: {"John": 2},
             2: {"likes": 3},
             3: {"Bill": 4}
            }
      }

In [ ]:
sentence = ["John", "likes", "Bill"]
accepts(sentence, fsa)

In [ ]:
sentence = ["John", "likes", "Sue"]
accepts(sentence, fsa)

This is another example of the importance of good data structures.
A bad data structure will make it very hard to write elegant and efficient code.
With a good data structure, good code comes about almost by itself.

Of course the data structure above could still be improved on.
Dictionaries consume quite a bit of memory, so a professional software engineer might want to design a new data structure from scratch just for transitions.
A very different route uses a mathematical theorem to translate recognition by an FSA into an instance of Boolean matrix multiplication (fancy term, huh?).
This would take us too far, though --- if you're curious about this there's some advanced readings I can give you.

But there's still one thing that's annoying about the current setup:
Our function presupposes a very specific FSA, and any deviation from that will make it misbehave (e.g. if there is a set of initial states).
The function and the data structure for FSAs are so tightly tied together that it makes no sense to make the function with anything else.
Python provides us with the means to directly couple functions to the data structures they operate on: **classes**.
Check the next notebook for the gory details (don't worry, it's actually pretty simple).

## Bullet point summary

- Finite-state automata are a powerful alternative to n-gram models, but their implementation is trickier.
- Good code needs good data structures.
- If possible, avoid iterating over the same thing multiple times (sometimes it cannot be avoided, though).
- Try to avoid recomputing results:
    - memoization if necessary
    - but a good data structure is preferable